# modulos y rutas

In [ ]:
# Importar librerías necesarias
import os  # Para trabajar con rutas del sistema
import pandas as pd  # Para trabajar con dataframes y archivos Excel
import numpy as np  # Para operaciones numéricas
import re  # Para validación de correos electrónicos
from datetime import datetime  # Para trabajar con fechas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from datetime import datetime
import sys
from pathlib import Path

## DETECCIÓN AUTOMÁTICA DEL PROYECTO

In [ ]:
# ========================
# 🔧 DETECCIÓN AUTOMÁTICA DEL PROYECTO
# ========================
notebook_path = Path.cwd()
project_root = notebook_path

while not (project_root / 'src').exists() and project_root != project_root.parent:
    project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Proyecto detectado: {project_root}")

## 🚩CONFIGURACIÓN (SOLO EDITAR AQUÍ)

In [ ]:
# ========================
# 📅 CONFIGURACIÓN (SOLO EDITAR AQUÍ)
# ========================
# Fecha principal del proceso
Fecha = "15-02-2026"  # Formato: DD-MM-YYYY

# 📅 Fechas específicas de archivos (editar según disponibilidad)
FECHAS_ARCHIVOS = {
    "ms_sie": None,           # None = usa fecha principal
    "ms_adres": "10-02-2026", # Fecha específica del maestro ADRES
    "sisben": None,           # None = usa fecha principal
    "pr": None,               # None = usa fecha principal (formato nombre archivo)
}

# Información del informe CTO (editar según corresponda)
INFORME_CTO = {
    "codigo": "CTO 102.2026",
    "numero": "#02",
    "actividad": "12 ACTIVIDAD"
}

# Nombre del archivo de Presuntos Repetidos (PR)
# Opciones comunes: "SEGUIMIENTO NACIMIENTOS (version 1).xlsb.xlsx" o formato con fecha
NOMBRE_ARCHIVO_PR = "SEGUIMIENTO NACIMIENTOS (version 1).xlsb.xlsx"
# Si el archivo PR usa formato de fecha: f"{fecha_pr['dia']}-{fecha_pr['mes_numero']}-{fecha_pr['año']}.xlsx"

# Nombre del archivo SISBEN (si tiene patrón específico)
# Dejar None para usar patrón por defecto "DNP-SISBEN-CO-XXXXXXXX.xlsx"
NOMBRE_ARCHIVO_SISBEN = "DNP-SISBEN-CO-0000086146.xlsx"  # None O especificar: "DNP-SISBEN-CO-0000086146.xlsx"

 ## CONSTRUCCIÓN AUTOMÁTICA DE FECHAS

In [ ]:
# ========================
# 🗓️ CONSTRUCCIÓN AUTOMÁTICA DE FECHAS
# ========================
def parsear_fecha(fecha_str):
    """Convierte fecha DD-MM-YYYY a componentes individuales"""
    fecha_obj = datetime.strptime(fecha_str, "%d-%m-%Y")
    return {
        "obj": fecha_obj,
        "año": str(fecha_obj.year),
        "mes_numero": f"{fecha_obj.month:02d}",
        "dia": f"{fecha_obj.day:02d}",
        "mes_nombre": {
            1:"Enero", 2:"Febrero", 3:"Marzo", 4:"Abril", 5:"Mayo", 6:"Junio",
            7:"Julio", 8:"Agosto", 9:"Septiembre", 10:"Octubre", 11:"Noviembre", 12:"Diciembre"
        }[fecha_obj.month],
        "fecha_str": fecha_str,
        "año_mes": None  # Para formato "2025-02"
    }

# Parsear fecha principal
fecha_principal = parsear_fecha(Fecha)
año = fecha_principal["año"]
mes_numero = fecha_principal["mes_numero"]
dia = fecha_principal["dia"]
mes_nombre = fecha_principal["mes_nombre"]

# Parsear fechas de archivos
fecha_sie = parsear_fecha(FECHAS_ARCHIVOS.get("ms_sie") or Fecha)
fecha_adres = parsear_fecha(FECHAS_ARCHIVOS.get("ms_adres") or Fecha)
fecha_sisben = parsear_fecha(FECHAS_ARCHIVOS.get("sisben") or Fecha)
fecha_pr = parsear_fecha(FECHAS_ARCHIVOS.get("pr") or Fecha)



# ========================
# 📂 CONSTRUCCIÓN AUTOMÁTICA DE RUTAS
# ========================
# Detectar usuario automáticamente
usuario = os.getlogin()
print(f"👤 Usuario detectado: {usuario}")

# Base OneDrive
onedrive_base = Path(f"C:/Users/{usuario}/OneDrive - 891856000_CAPRESOCA E P S")

if not onedrive_base.exists():
    print(f"⚠️  Advertencia: OneDrive no encontrado en: {onedrive_base}")

# ========================
# 🗂️ RUTAS DE ARCHIVOS DE ENTRADA
# ========================
capresoca_base = onedrive_base / "Capresoca/AlmostClear"

# MS del SIE
R_MS_SIe = capresoca_base / "SIE/Aseguramiento/ms_sie" / f"Reporte_Validación Archivos Maestro_{fecha_sie['año']}_{fecha_sie['mes_numero']}_{fecha_sie['dia']}.csv"

# Maestro ADRES Subsidiado
R_MS_ADRES = capresoca_base / "Procesos BDUA/Subsidiados/Maestro/MS" / f"{fecha_adres['año']}" / f"EPS025MS00{fecha_adres['dia']}{fecha_adres['mes_numero']}{fecha_adres['año']}.TXT"

# Código DANE
R_Cod_DANE = capresoca_base / "Constantes/Departamentos.txt"

# Presuntos Repetidos (PR)
if NOMBRE_ARCHIVO_PR:
    archivo_pr = NOMBRE_ARCHIVO_PR
else:
    archivo_pr = f"{fecha_pr['dia']}-{fecha_pr['mes_numero']}-{fecha_pr['año']}.xlsx"

R_PR = (onedrive_base / "Escritorio/Yesid Rincón Z/informes" / año / INFORME_CTO["codigo"] / 
        f"{INFORME_CTO['codigo'].replace(' ', '')} Informe  {INFORME_CTO['numero']}" / 
        INFORME_CTO["actividad"] / "Bases de datos notificaciones telefonicas" / archivo_pr)

# SISBEN
if NOMBRE_ARCHIVO_SISBEN:
    archivo_sisben = NOMBRE_ARCHIVO_SISBEN
else:
    # Si no se especifica, buscar cualquier archivo que coincida con el patrón
    ruta_sisben_dir = capresoca_base / "SISBEN" / fecha_sisben['año'] / f"{fecha_sisben['mes_numero']}_{fecha_sisben['mes_nombre']}"
    archivo_sisben = "DNP-SISBEN-CO-*.xlsx"  # Patrón para búsqueda

R_Sisben = capresoca_base / "SISBEN" / fecha_sisben['año'] / f"{fecha_sisben['mes_numero']}_{fecha_sisben['mes_nombre']}" / archivo_sisben

# ========================
# 📤 RUTAS DE SALIDA
# ========================
base_salida = (onedrive_base / "Escritorio/Yesid Rincón Z/informes" / año / INFORME_CTO["codigo"] / 
               f"{INFORME_CTO['codigo'].replace(' ', '')} Informe  {INFORME_CTO['numero']}" / 
               INFORME_CTO["actividad"] / "Bases de datos notificaciones telefonicas")

# Definir todas las rutas de salida
Ruta_Salida_Actualización_Datos = base_salida / "NOTIFICAR ACTUALIZACION DE DATOS" / f"Actualización de datos_{Fecha}.xlsx"
Ruta_Salida_Actualización_Documento = base_salida / "ACTUALIZACION DE DOCUMENTO" / f"Actualización_documento-{Fecha}.xlsx"
Ruta_Salida_No_Sisben = base_salida / "USUARIOS ACTIVOS SIN SISBEN (IV)" / f"No Sisben-{Fecha}.xlsx"
R_resumen_SIE = base_salida / "resumen SIE.xlsx"
R_MS_SIE_Correo = base_salida / "Validado SIE.xlsx"
R_Sisben_OtroMunicipio = base_salida / "NOTIFICACION SISBEN EN OTRO MUNICIPIO" / f"Sisben otro municipio-{Fecha}.xlsx"

# ========================
# ✅ VERIFICACIÓN DE ARCHIVOS
# ========================
print(f"\n{'='*70}")
print(f"🔍 VERIFICACIÓN DE RUTAS Y ARCHIVOS")
print(f"{'='*70}")
print(f"👤 Usuario: {usuario}")
print(f"📅 Fecha Principal: {Fecha} ({mes_nombre} {año})")
print(f"📅 Fecha MS SIE: {fecha_sie['fecha_str']}")
print(f"📅 Fecha MS ADRES: {fecha_adres['fecha_str']} (carpeta: {fecha_adres['año']})")
print(f"📅 Fecha SISBEN: {fecha_sisben['fecha_str']}")
print(f"📋 Informe CTO: {INFORME_CTO['codigo']} {INFORME_CTO['numero']}")
print(f"📁 Actividad: {INFORME_CTO['actividad']}")

# Verificar archivos de entrada
rutas_entrada = {
    "📋 MS SIE": R_MS_SIe,
    "📊 Maestro ADRES": R_MS_ADRES,
    "🗺️  Código DANE": R_Cod_DANE,
    "📄 Presuntos Repetidos": R_PR,
}

# Para SISBEN, verificar si es patrón o archivo específico
if "*" in str(R_Sisben):
    # Buscar archivos que coincidan con el patrón
    sisben_dir = R_Sisben.parent
    if sisben_dir.exists():
        archivos_sisben = list(sisben_dir.glob(R_Sisben.name))
        if archivos_sisben:
            R_Sisben = archivos_sisben[0]  # Tomar el primero encontrado
            rutas_entrada["📈 SISBEN"] = R_Sisben
        else:
            rutas_entrada["📈 SISBEN (⚠️ No encontrado)"] = sisben_dir / R_Sisben.name
    else:
        rutas_entrada["📈 SISBEN (⚠️ Directorio no existe)"] = sisben_dir
else:
    rutas_entrada["📈 SISBEN"] = R_Sisben

print(f"\n📄 Archivos de entrada:")
todas_existen = True
for nombre, ruta in rutas_entrada.items():
    existe = ruta.exists() if isinstance(ruta, Path) else False
    simbolo = "✅" if existe else "❌"
    todas_existen = todas_existen and existe
    print(f"  {simbolo} {nombre}")
    if isinstance(ruta, Path):
        print(f"     └─ {ruta.name}")
        if not existe:
            print(f"     └─ ⚠️  {ruta}")

# Verificar rutas de salida
print(f"\n📤 Archivos de salida:")
rutas_salida = {
    "Actualización Datos": Ruta_Salida_Actualización_Datos,
    "Actualización Documento": Ruta_Salida_Actualización_Documento,
    "No Sisben": Ruta_Salida_No_Sisben,
    "Resumen SIE": R_resumen_SIE,
    "Validado SIE": R_MS_SIE_Correo,
    "Sisben Otro Municipio": R_Sisben_OtroMunicipio
}

for nombre, ruta in rutas_salida.items():
    dir_existe = ruta.parent.exists()
    simbolo = "📁" if dir_existe else "⚠️"
    print(f"  {simbolo} {nombre}")
    print(f"     └─ {ruta.name}")
    if not dir_existe:
        print(f"     └─ ⚠️  Directorio: {ruta.parent}")
        print(f"     └─ 📝 Se creará al guardar")

print(f"{'='*70}\n")

if not todas_existen:
    print("⚠️  ADVERTENCIA: Algunos archivos de entrada no existen")
else:
    print("✅ Todos los archivos de entrada verificados\n")

# ========================
# 🎯 RESUMEN DE CONFIGURACIÓN
# ========================
print("🎯 Variables de rutas disponibles:")
print(f"   📥 Entrada:")
print(f"      - R_MS_SIe")
print(f"      - R_MS_ADRES")
print(f"      - R_Cod_DANE")
print(f"      - R_PR")
print(f"      - R_Sisben")
print(f"   📤 Salida:")
print(f"      - Ruta_Salida_Actualización_Datos")
print(f"      - Ruta_Salida_Actualización_Documento")
print(f"      - Ruta_Salida_No_Sisben")
print(f"      - R_resumen_SIE")
print(f"      - R_MS_SIE_Correo")
print(f"      - R_Sisben_OtroMunicipio\n")

# Datafarmes

In [ ]:
# Cargar el archivo .txt en un DataFrame
df_cod_dane = pd.read_csv(R_Cod_DANE, sep=';', encoding='UTF-8')
df_ms_adres = pd.read_csv(R_MS_ADRES, sep=',', encoding='ansi', header=None)
df_ms_SIE = pd.read_csv(R_MS_SIe, sep=';', encoding='ansi')
# Cargar los datos de la hoja "PR" del archivo Excel en la ruta R_PR
df_PR = pd.read_excel(R_PR, sheet_name="PR", usecols=["T_DOC", "N_DOC"], header=0)
df_Sisben = pd.read_excel(R_Sisben, sheet_name="Resultado")

In [ ]:
# Formatear la columna 'CODIGO' de df_cod_dane para que tenga 5 dígitos
df_cod_dane['CODIGO'] = df_cod_dane['CODIGO'].astype(int).apply(lambda x: f"{x:05d}")

In [ ]:
df_ms_SIE = df_ms_SIE[['tipo_documento', 'numero_identificacion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']]
cols = ['celular', 'telefono_1', 'telefono_2']
df_ms_SIE[cols] = df_ms_SIE[cols].apply(lambda col: col.str.replace('[- ]', '', regex=True))
def valid_phone(val):
    val_str = str(val)
    if val_str.isdigit() and len(val_str) == 10 and val_str.startswith('3'):
        return val_str
    return ''

df_ms_SIE[cols] = df_ms_SIE[cols].applymap(valid_phone)

In [ ]:
# ========================
# 🔧 VALIDACIÓN DE CORREOS ELECTRÓNICOS CON AUDITORÍA (VERSIÓN MEJORADA)
# ========================

def validar_correo_electronico_con_auditoria(email):
    """
    Valida correos electrónicos y retorna tupla (correo_validado, motivo_rechazo)
    Acepta caracteres latinos (ñ, tildes) según RFC 6531
    Rechaza dominios mal escritos y correos temporales
    Normaliza a minúsculas para mejor agrupación
    """
    # 1. Validar nulos y vacíos
    if pd.isnull(email) or str(email).strip() == "":
        return ("", "Correo nulo o vacío")
    
    email_original = str(email).strip()
    # 🆕 NORMALIZAR A MINÚSCULAS desde el inicio
    email_lower = email_original.lower()
    
    # 2. Validar estructura básica de email (permite caracteres latinos)
    patron_email = r'^[a-zA-ZáéíóúÁÉÍÓÚñÑüÜ0-9_.+-]+@[a-zA-ZáéíóúÁÉÍÓÚñÑüÜ0-9-]+\.[a-zA-Z]{2,}$'
    if not re.match(patron_email, email_lower):
        return ("", f"Estructura inválida: {email_original}")
    
    # 3. Validar que tenga @ (redundante pero por claridad)
    if '@' not in email_lower:
        return ("", f"Sin símbolo @: {email_original}")
    
    # 4. Extraer usuario y dominio
    usuario, dominio_completo = email_lower.split('@', 1)
    
    # 5. Validar que el dominio tenga al menos un punto
    if '.' not in dominio_completo:
        return ("", f"Dominio sin extensión: {email_original}")
    
    # ========================
    # 🆕 6. VALIDAR DOMINIOS MAL ESCRITOS (ERRORES DE DIGITACIÓN)
    # ========================
    dominios_mal_escritos = {
        # Gmail mal escrito
        'gamil.com': 'gmail.com',
        'gmial.com': 'gmail.com',
        'gmai.com': 'gmail.com',
        'gmil.com': 'gmail.com',
        'gmaul.com': 'gmail.com',
        'gmal.com': 'gmail.com',
        'gnail.com': 'gmail.com',
        'gimail.com': 'gmail.com',
        'gemail.com': 'gmail.com',
        'gmeil.com': 'gmail.com',
        'gmaill.com': 'gmail.com',
        'gmailcom': 'gmail.com',  # Sin punto
        'gmail.co': 'gmail.com',
        'gmail.con': 'gmail.com',
        'gmail.vom': 'gmail.com',
        'gmail.cpm': 'gmail.com',
        'gamail.com': 'gmail.com',  # 🆕 NUEVO - Error común
        
        # Hotmail mal escrito
        'hotmial.com': 'hotmail.com',
        'hotmeil.com': 'hotmail.com',
        'hotmal.com': 'hotmail.com',
        'hotmil.com': 'hotmail.com',
        'homail.com': 'hotmail.com',
        'hoymail.com': 'hotmail.com',
        'hotmailcom': 'hotmail.com',  # Sin punto
        'hotmail.co': 'hotmail.com',
        'hotmail.con': 'hotmail.com',
        'hotmail.vom': 'hotmail.com',
        'hotmali.com': 'hotmail.com',
        'hotmaul.com': 'hotmail.com',
        
        # Outlook mal escrito
        'outlok.com': 'outlook.com',
        'outlock.com': 'outlook.com',
        'outluk.com': 'outlook.com',
        'otlook.com': 'outlook.com',
        'outlookcom': 'outlook.com',  # Sin punto
        'outlook.co': 'outlook.com',
        'outlook.con': 'outlook.com',
        'outlool.com': 'outlook.com',
        'outloock.com': 'outlook.com',
        
        # Yahoo mal escrito
        'yaho.com': 'yahoo.com',
        'yahooo.com': 'yahoo.com',
        'yhaoo.com': 'yahoo.com',
        'yajoo.com': 'yahoo.com',
        'yahho.com': 'yahoo.com',
        'yahoocom': 'yahoo.com',  # Sin punto
        'yahoo.co': 'yahoo.com',
        'yahoo.con': 'yahoo.com',
        'yahoo.es.com': 'yahoo.es',
        
        # Otros errores comunes
        'live.co': 'live.com',
        'icloud.co': 'icloud.com',
        'protonmail.co': 'protonmail.com',
    }
    
    if dominio_completo in dominios_mal_escritos:
        dominio_correcto = dominios_mal_escritos[dominio_completo]
        return ("", f"Dominio mal escrito '{dominio_completo}' (debería ser '{dominio_correcto}'): {email_original}")
    
    # ========================
    # 🆕 7. LISTA NEGRA DE DOMINIOS TEMPORALES/DESECHABLES
    # ========================
    dominios_temporales = [
        'gufum.com',         # Correos temporales
        '10minutemail.com',  # Correos temporales
        'guerrillamail.com', # Correos temporales
        'mailinator.com',    # Correos temporales
        'temp-mail.org',     # Correos temporales
        'throwaway.email',   # Correos temporales
        'tempmail.com',      # Correos temporales
        'yopmail.com',       # Correos temporales
        'maildrop.cc',       # Correos temporales
        'trashmail.com',     # Correos temporales
    ]
    
    if dominio_completo in dominios_temporales:
        return ("", f"Dominio de correo temporal/desechable '{dominio_completo}': {email_original}")
    
    # ========================
    # 8. Lista negra de correos específicos
    # ========================
    correos_invalidos = [
        'actualizar@actualizar.com',
        'notiene@gmail.com',
        'notiene@gamil.com',
        '00@hotmail.com',
        'sincorreo@gmail.com',
        'noaplica@gmail.com',
        'ninguno@gmail.com',
        'actualizar@hotmail.com',
        'actualizar@outlook.com'
    ]
    if email_lower in correos_invalidos:
        return ("", f"Lista negra específica: {email_original}")
    
    # ========================
    # 9. Validar palabras clave inválidas en TODO el correo
    # ========================
    palabras_invalidas = {
        r'\bactualizar\b': 'actualizar',
        r'\bactualiza\b': 'actualiza',
        r'\btraslado\b': 'traslado',
        r'\bsat\b': 'sat',
        r'\btiene\b': 'tiene',
        r'\bafilia\b': 'afilia',
        r'\bnoaplica\b': 'noaplica',
        r'\bsincorreo\b': 'sincorreo',
        r'\bninguno\b': 'ninguno',
        r'\bnoexiste\b': 'noexiste',
        r'\bnotiene\b': 'notiene',
        r'\bsinmail\b': 'sinmail',
        r'\bnoemail\b': 'noemail',
        r'\bsin\s*correo\b': 'sin correo',
        r'\bno\s*tiene\b': 'no tiene'
    }
    
    for patron, palabra in palabras_invalidas.items():
        if re.search(patron, email_lower):
            return ("", f"Palabra inválida '{palabra}': {email_original}")
    
    # ========================
    # 10. Validar dominios inválidos
    # ========================
    dominios_invalidos = {
        r'\bactualizar\.': 'actualizar',
        r'\bnotiene\.': 'notiene',
        r'\bsincorreo\.': 'sincorreo',
        r'\bnoaplica\.': 'noaplica',
        r'\bninguno\.': 'ninguno',
        r'\bnoexiste\.': 'noexiste',
        r'\bafilia\.': 'afilia',
        r'\btraslado\.': 'traslado',
        r'\bsat\.': 'sat',
        r'\btiene\.': 'tiene',
        r'\bsinmail\.': 'sinmail',
        r'\bnoemail\.': 'noemail'
    }
    
    for patron_dominio, dominio in dominios_invalidos.items():
        if re.search(patron_dominio, dominio_completo):
            return ("", f"Dominio inválido '@{dominio}.*': {email_original}")
    
    # ========================
    # 11. Descartar si el usuario es solo números
    # ========================
    if usuario.isdigit():
        return ("", f"Usuario solo numérico: {email_original}")
    
    # ========================
    # 12. Validar longitud mínima del usuario (al menos 2 caracteres)
    # ========================
    if len(usuario) < 2:
        return ("", f"Usuario muy corto: {email_original}")
    
    # ========================
    # 13. Validar que la extensión del dominio sea válida (al menos 2 caracteres)
    # ========================
    extension = dominio_completo.split('.')[-1]
    if len(extension) < 2:
        return ("", f"Extensión de dominio inválida: {email_original}")
    
    # ✅ Si pasó todas las validaciones, retornar EN MINÚSCULAS
    return (email_lower, "")


# ========================
# 📊 APLICAR VALIDACIÓN CON AUDITORÍA
# ========================
print("🔍 Iniciando validación de correos electrónicos con auditoría...")
print(f"📋 Total de registros: {len(df_ms_SIE):,}\n")

# Crear copia del DataFrame original para auditoría
df_correos_original = df_ms_SIE[['tipo_documento', 'numero_identificacion', 'correo_electronico']].copy()

# Contar correos antes de la validación
correos_antes = df_ms_SIE['correo_electronico'].notna().sum()

# Aplicar la validación y guardar resultados
resultados = df_ms_SIE['correo_electronico'].apply(validar_correo_electronico_con_auditoria)

# Separar correos validados y motivos de rechazo
df_ms_SIE['correo_electronico'] = resultados.apply(lambda x: x[0])
df_correos_original['correo_validado'] = resultados.apply(lambda x: x[0])
df_correos_original['motivo_rechazo'] = resultados.apply(lambda x: x[1])

# Contar correos después de la validación
correos_despues = df_ms_SIE['correo_electronico'].ne("").sum()
correos_eliminados = correos_antes - correos_despues

print(f"✅ Validación completada:")
print(f"   📧 Correos antes: {correos_antes:,}")
print(f"   ✅ Correos válidos: {correos_despues:,}")
print(f"   ❌ Correos eliminados: {correos_eliminados:,}")
print(f"   📊 Tasa de validez: {(correos_despues/correos_antes*100):.2f}%")
print(f"   🔡 Todos los correos normalizados a minúsculas\n")

# ========================
# 📋 CREAR DATAFRAME DE CORREOS RECHAZADOS
# ========================
df_correos_rechazados = df_correos_original[df_correos_original['motivo_rechazo'] != ''].copy()

# Reorganizar columnas
df_correos_rechazados.rename(columns={'correo_electronico': 'correo_original'}, inplace=True)
df_correos_rechazados = df_correos_rechazados[['tipo_documento', 'numero_identificacion', 'correo_original', 'motivo_rechazo']]

print(f"📋 DataFrame de correos rechazados creado:")
print(f"   📊 Total de correos rechazados: {len(df_correos_rechazados):,}")
print(f"   📁 Variable: df_correos_rechazados\n")

# ========================
# 📊 ANÁLISIS DE MOTIVOS DE RECHAZO
# ========================
print("="*80)
print("📊 ANÁLISIS DE MOTIVOS DE RECHAZO")
print("="*80)

# Agrupar por motivo de rechazo
resumen_rechazos = df_correos_rechazados['motivo_rechazo'].value_counts()

print(f"\n🔍 Top 15 motivos de rechazo más comunes:\n")
for idx, (motivo, count) in enumerate(resumen_rechazos.head(15).items(), 1):
    porcentaje = (count / len(df_correos_rechazados)) * 100 if len(df_correos_rechazados) > 0 else 0
    print(f"   {idx:2d}. {count:6,} ({porcentaje:5.2f}%) - {motivo}")

print("\n" + "="*80)

# ========================
# 📝 MUESTRA DE CORREOS RECHAZADOS
# ========================
if len(df_correos_rechazados) > 0:
    print("\n📝 Muestra de correos rechazados (primeros 10):")
    print("-" * 80)
    pd.set_option('display.max_colwidth', 50)
    print(df_correos_rechazados.head(10).to_string(index=False))
    print("-" * 80 + "\n")
else:
    print("\n✅ ¡EXCELENTE! No hay correos rechazados\n")

print("✅ DataFrame 'df_correos_rechazados' listo para usar en reportes posteriores\n")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ========================
# 📊 ANÁLISIS DE CORREOS ELECTRÓNICOS
# ========================

print("\n" + "="*60)
print("📊 ANÁLISIS DE CORREOS ELECTRÓNICOS")
print("="*60)

# 1️⃣ Contar correos válidos vs vacíos
correos_validos = df_ms_SIE['correo_electronico'].ne("").sum()
correos_vacios = df_ms_SIE['correo_electronico'].eq("").sum()
total_registros = len(df_ms_SIE)

print(f"\n✅ Total de registros: {total_registros:,}")
print(f"📧 Correos válidos: {correos_validos:,} ({correos_validos/total_registros*100:.1f}%)")
print(f"❌ Correos vacíos: {correos_vacios:,} ({correos_vacios/total_registros*100:.1f}%)")

# 2️⃣ Gráfico de barras: Correos válidos vs vacíos
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico 1: Conteo
categorias = ['Válidos', 'Vacíos']
valores = [correos_validos, correos_vacios]
colores = ['#5B9BD5', '#ED7D31']

ax[0].bar(categorias, valores, color=colores)
ax[0].set_ylabel('Cantidad de registros')
ax[0].set_title('Correos Electrónicos: Válidos vs Vacíos')
ax[0].set_ylim(0, max(valores) * 1.1)

# Agregar valores en las barras
for i, v in enumerate(valores):
    ax[0].text(i, v + max(valores)*0.02, f'{v:,}\n({v/total_registros*100:.1f}%)', 
               ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Porcentaje (Pie)
ax[1].pie(valores, labels=categorias, autopct='%1.1f%%', colors=colores, startangle=90)
ax[1].set_title('Distribución Porcentual')

plt.tight_layout()
plt.show()

# 3️⃣ Top 10 dominios de correo más comunes
print("\n📈 Top 10 dominios de correo más utilizados:")
print("-" * 60)

correos_validos_df = df_ms_SIE[df_ms_SIE['correo_electronico'].ne("")].copy()

if len(correos_validos_df) > 0:
    # Extraer el dominio (después del @)
    correos_validos_df['dominio'] = correos_validos_df['correo_electronico'].str.split('@').str[1]
    
    # Contar los dominios más frecuentes
    top_dominios = correos_validos_df['dominio'].value_counts().head(10)
    
    for idx, (dominio, count) in enumerate(top_dominios.items(), 1):
        porcentaje = (count / len(correos_validos_df)) * 100
        print(f"   {idx:2d}. {dominio:25s} - {count:6,} correos ({porcentaje:5.2f}%)")
    
    # Gráfico de dominios
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(top_dominios)), top_dominios.values, color='#5B9BD5')
    plt.yticks(range(len(top_dominios)), top_dominios.index)
    plt.xlabel('Cantidad de correos')
    plt.title('Top 10 Dominios de Correo Electrónico')
    plt.gca().invert_yaxis()
    
    # Agregar valores en las barras
    for i, v in enumerate(top_dominios.values):
        plt.text(v + max(top_dominios.values)*0.01, i, f'{v:,}', 
                va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  No hay correos válidos para analizar dominios")

# 4️⃣ Ejemplos de correos válidos aleatorios
print("\n📝 Muestra de 10 correos válidos (aleatorios):")
print("-" * 60)

if len(correos_validos_df) > 0:
    muestra = correos_validos_df['correo_electronico'].sample(min(10, len(correos_validos_df)))
    for idx, correo in enumerate(muestra, 1):
        print(f"   {idx:2d}. {correo}")
else:
    print("⚠️  No hay correos válidos para mostrar")

print("\n" + "="*60 + "\n")

In [ ]:
empty_count = df_ms_SIE['correo_electronico'].eq("").sum()
non_empty_count = df_ms_SIE['correo_electronico'].ne("").sum()

print("Cantidad de registros vacíos en 'correo_electronico':", empty_count)
print("Cantidad de registros no vacíos en 'correo_electronico':", non_empty_count)

In [ ]:
empty_count = df_ms_SIE['correo_electronico'].eq("").sum()
non_empty_count = df_ms_SIE['correo_electronico'].ne("").sum()

print("Cantidad de registros vacíos en 'correo_electronico':", empty_count)
print("Cantidad de registros no vacíos en 'correo_electronico':", non_empty_count)

In [ ]:
print(df_ms_SIE[df_ms_SIE['numero_identificacion'] == '1121872381'])

In [ ]:
# Calcular el total de registros de df_ms_SIE
total_registros = df_ms_SIE.shape[0]

tabla = []
columnas = ['celular', 'telefono_1', 'telefono_2', 'correo_electronico']

for col in columnas:
    # Calcular la cantidad de registros vacíos y con datos
    vacios = df_ms_SIE[col].apply(lambda x: pd.isnull(x) or x == "").sum()
    con_dato = total_registros - vacios
    # Calcular porcentajes
    pct_con_dato = (con_dato / total_registros) * 100
    pct_vacios = (vacios / total_registros) * 100
    tabla.append({
        'Columna': col,
        'Total': total_registros,
        'Datos (Cantidad)': con_dato,
        '% Datos': f"{pct_con_dato:.2f}%",
        'Vacíos (Cantidad)': vacios,
        '% Vacíos': f"{pct_vacios:.2f}%"
    })

# Convertir a DataFrame y mostrar la tabla
tabla_df = pd.DataFrame(tabla)
print("Cantidad total de registros en df_ms_SIE:", total_registros)
print(tabla_df)

In [ ]:
with pd.ExcelWriter(R_resumen_SIE, engine='xlsxwriter') as writer:
    # Escribir tabla_df en la hoja "Resumen" empezando en la fila 2 (fila 1 para cabecera)
    tabla_df.to_excel(writer, sheet_name='Resumen', index=False, startrow=1)

    workbook  = writer.book
    worksheet = writer.sheets['Resumen']

    # Formato para la cabecera: negrita, ajuste de texto y color de fondo
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'vcenter',
        'fg_color': '#DCE6F1',
        'border': 1
    })

    # Aplicar el formato a la cabecera
    for col_num, value in enumerate(tabla_df.columns.values):
        worksheet.write(1, col_num, value, header_format)

    # Definir el rango de datos (la tabla comienza en la fila 2, por lo que cabe recalcular índices)
    nrows = len(tabla_df)
    first_data_row = 2
    last_data_row = first_data_row + nrows - 1

    # Crear un gráfico de columnas para comparar "Datos (Cantidad)" y "Vacíos (Cantidad)"
    chart = workbook.add_chart({'type': 'column'})

    # Agregar serie para "Datos (Cantidad)" (columna 3, índice 2)
    chart.add_series({
        'name':       'Datos (Cantidad)',
        'categories': ['Resumen', first_data_row, 0, last_data_row, 0],
        'values':     ['Resumen', first_data_row, 2, last_data_row, 2],
        'fill':       {'color': '#5B9BD5'},
    })

    # Agregar serie para "Vacíos (Cantidad)" (columna 5, índice 4)
    chart.add_series({
        'name':       'Vacíos (Cantidad)',
        'categories': ['Resumen', first_data_row, 0, last_data_row, 0],
        'values':     ['Resumen', first_data_row, 4, last_data_row, 4],
        'fill':       {'color': '#ED7D31'},
    })

    chart.set_title({'name': 'Resumen de Datos vs Vacíos'})
    chart.set_x_axis({'name': 'Columna'})
    chart.set_y_axis({'name': 'Cantidad'})
    chart.set_style(10)

    # Insertar el gráfico en la hoja (ubicado en la celda H2)
    worksheet.insert_chart('H2', chart, {'x_scale': 1.0, 'y_scale': 1.0})

    # Agregar una breve descripción debajo de la tabla y del gráfico
    description = (
        "La tabla muestra, por cada columna evaluada, el total de registros, "
        "la cantidad de registros con datos y la cantidad de registros vacíos. "
        "La gráfica ilustra visualmente la comparación entre los registros con datos "
        "y aquellos que están vacíos, facilitando la identificación de áreas que requieran atención."
    )
    worksheet.write(last_data_row + 3, 0, description)

In [ ]:
df_ms_SIE.columns

In [ ]:
# Crear la nueva columna "municipio" con el formato deseado:
df_ms_adres['municipio'] = (
    df_ms_adres[23].astype(int).apply(lambda x: f"{x:02d}") +
    df_ms_adres[24].astype(int).apply(lambda x: f"{x:03d}")
)

# Eliminar las columnas 23 y 24
df_ms_adres.drop(columns=[23, 24], inplace=True)

df_ms_adres = df_ms_adres.merge(
    df_cod_dane[['CODIGO', 'Nombre Departamento', 'Nombre Municipio']],
    left_on='municipio',
    right_on='CODIGO',
    how='left'
)
df_ms_adres.drop('CODIGO', axis=1, inplace=True)

# Convertir la columna 5 de df_ms_adres a string para asegurar la coincidencia
df_ms_adres[5] = df_ms_adres[5].astype(str)
df_ms_SIE['numero_identificacion'] = df_ms_SIE['numero_identificacion'].astype(str)

# Seleccionar las columnas a unir de df_ms_SIE
cols_to_merge = ['numero_identificacion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']

# Realizar el merge usando la columna "numero_identificacion" de df_ms_SIE y la columna 5 de df_ms_adres
df_ms_adres = df_ms_adres.merge(df_ms_SIE[cols_to_merge],
                                                      left_on=5,
                                                      right_on='numero_identificacion',
                                                      how='left')
# Si ya no se necesita la columna duplicada, se puede eliminar
df_ms_adres.drop('numero_identificacion', axis=1, inplace=True)

In [ ]:
# Guardar df_ms_SIE en la ruta especificada como un archivo Excel
df_ms_SIE.to_excel(R_MS_SIE_Correo, index=False)

In [ ]:
df_ms_adres

# 1. NOTIFICAR ACTUALIZACION DE DATOS

Se organiza la base de datos de los usuarios sin correo para notivicar via mensaje de texto para que actualicen un correo y las personas sin telefono pero con correo electronico para que actualicen numeros de telefono.

In [ ]:
# Filtrar el DataFrame según las condiciones
Df_Actualizacion_Datos = df_ms_adres[
    (df_ms_adres[2].isnull() | (df_ms_adres[2] == '')) &  # Columna 2 vacía o nula
    (df_ms_adres[33] == 'AC') &  # Columna 33 igual a 'AC'
    (df_ms_adres[42] == 0)  # Columna 42 igual a 0
]
#Df_Actualizacion_Datos = Df_Actualizacion_Datos.iloc[:, list(range(4, 12)) + ['Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']]

# Obtener los nombres de las columnas correspondientes a los índices del 4 al 11
cols_por_indice = Df_Actualizacion_Datos.columns[4:12].tolist()
# Combinar con las columnas adicionales
columnas_finales = cols_por_indice + ['Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']
Df_Actualizacion_Datos = Df_Actualizacion_Datos.loc[:, columnas_finales]

In [ ]:
# Filtrar los registros donde todas las columnas de contacto están vacías o nulas
df_sin_datos = Df_Actualizacion_Datos[
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2', 'correo_electronico']].isnull().all(axis=1) |
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2', 'correo_electronico']].eq("").all(axis=1)
]

# Filtrar los registros donde al menos una de las columnas de teléfono no está vacía y el correo está vacío
df_telefonos = Df_Actualizacion_Datos[
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].notnull().any(axis=1) &
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].ne("").any(axis=1) &
    (Df_Actualizacion_Datos['correo_electronico'].isnull() | Df_Actualizacion_Datos['correo_electronico'].eq(""))
]

# Filtrar los registros donde todas las columnas de teléfono están vacías y el correo no está vacío
df_correo = Df_Actualizacion_Datos[
    (Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].isnull().all(axis=1) |
     Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].eq("").all(axis=1)) &
    Df_Actualizacion_Datos['correo_electronico'].notnull() &
    Df_Actualizacion_Datos['correo_electronico'].str.strip().ne("")
]

# Filtrar los registros donde al menos una de las columnas de teléfono no está vacía y el correo tampoco está vacío
df_ok = Df_Actualizacion_Datos[
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].notnull().any(axis=1) &
    Df_Actualizacion_Datos[['celular', 'telefono_1', 'telefono_2']].ne("").any(axis=1) &
    Df_Actualizacion_Datos['correo_electronico'].notnull() &
    Df_Actualizacion_Datos['correo_electronico'].ne("")
]

In [ ]:
# Verificar el valor del registro "1151188288" en la columna 5 y correo_electronico
registro = Df_Actualizacion_Datos[Df_Actualizacion_Datos[5] == "1151188288"]
print("Registro encontrado:")
print(registro[['correo_electronico']])

In [ ]:
df_ok

In [ ]:
import os

# Crear las tablas resumen
resumen1 = pd.DataFrame({
    "Grupo": ["Sin Datos", "Telefonos", "Correo", "OK"],
    "Cantidad": [len(df_sin_datos), len(df_telefonos), len(df_correo), len(df_ok)],
    "Descripción": [
        "Registros sin ningún dato de contacto",
        "Registros con al menos un teléfono pero sin correo",
        "Registros con correo pero sin teléfonos",
        "Registros con ambos datos completos"
    ]
})

municipios = Df_Actualizacion_Datos["Nombre Municipio"].unique()
lista = []
for mun in municipios:
    count_sin = df_sin_datos["Nombre Municipio"].eq(mun).sum()
    count_tel = df_telefonos["Nombre Municipio"].eq(mun).sum()
    count_cor = df_correo["Nombre Municipio"].eq(mun).sum()
    count_ok  = df_ok["Nombre Municipio"].eq(mun).sum()
    lista.append({
        "Nombre Municipio": mun,
        "Sin Datos": count_sin,
        "Telefonos": count_tel,
        "Correo": count_cor,
        "OK": count_ok
    })
resumen2 = pd.DataFrame(lista)

# Agregar una fila de totales a cada tabla
total_resumen1 = pd.DataFrame({
    "Grupo": ["Total"],
    "Cantidad": [resumen1["Cantidad"].sum()],
    "Descripción": [""]
})
resumen1 = pd.concat([resumen1, total_resumen1], ignore_index=True)

total_resumen2 = pd.DataFrame({
    "Nombre Municipio": ["Total"],
    "Sin Datos": [resumen2["Sin Datos"].sum()],
    "Telefonos": [resumen2["Telefonos"].sum()],
    "Correo": [resumen2["Correo"].sum()],
    "OK": [resumen2["OK"].sum()]
})
resumen2 = pd.concat([resumen2, total_resumen2], ignore_index=True)

# ========================
# 📋 PREPARAR TABLA RESUMEN DE CORREOS RECHAZADOS
# ========================
resumen_correos_rechazados = df_correos_rechazados['motivo_rechazo'].value_counts().reset_index()
resumen_correos_rechazados.columns = ['Motivo de Rechazo', 'Cantidad']
resumen_correos_rechazados['Porcentaje'] = (resumen_correos_rechazados['Cantidad'] / len(df_correos_rechazados) * 100).round(2)

# Agregar fila de total
total_correos_rechazados = pd.DataFrame({
    'Motivo de Rechazo': ['Total'],
    'Cantidad': [resumen_correos_rechazados['Cantidad'].sum()],
    'Porcentaje': [100.0]
})
resumen_correos_rechazados = pd.concat([resumen_correos_rechazados, total_correos_rechazados], ignore_index=True)

# Crear el Excel con los 4 dataframes y la hoja resumen.
with pd.ExcelWriter(Ruta_Salida_Actualización_Datos, engine='xlsxwriter') as writer:
    # Escribir cada dataframe en su respectiva hoja
    df_sin_datos.to_excel(writer, sheet_name="Sin_Datos", index=False)
    df_telefonos.to_excel(writer, sheet_name="Telefonos", index=False)
    df_correo.to_excel(writer, sheet_name="Correo", index=False)
    df_ok.to_excel(writer, sheet_name="OK", index=False)
    
    # ========================
    # 📋 NUEVA HOJA: AUDITORÍA DE CORREOS RECHAZADOS
    # ========================
    df_correos_rechazados.to_excel(writer, sheet_name="Auditoria_Correos", index=False)
    
    # Crear hoja Resumen y preparar formatos
    resumen_sheet_name = "Resumen"
    resumen_ws = writer.book.add_worksheet(resumen_sheet_name)
    
    # Create formats for header, cells and totales
    header_format = writer.book.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'middle',
        'align': 'center',
        'fg_color': '#DCE6F1',
        'border': 1
    })
    cell_format = writer.book.add_format({
        'border': 1,
        'valign': 'top'
    })
    total_cell_format = writer.book.add_format({
        'border': 1,
        'valign': 'top',
        'bold': True,
        'bg_color': '#FFD700'
    })
    
    # Ajustar ancho de columnas para la primera tabla
    resumen_ws.set_column('A:C', 25)
    
    # ========================
    # TABLA 1: Resumen por Grupo
    # ========================
    # Escribir encabezado de resumen1
    for col_num, value in enumerate(resumen1.columns.tolist()):
         resumen_ws.write(0, col_num, value, header_format)
    # Escribir datos de resumen1 (resaltando la fila de totales)
    for row_idx, row in resumen1.iterrows():
         fmt = total_cell_format if row_idx == len(resumen1)-1 else cell_format
         for col_idx, cell_val in enumerate(row.tolist()):
             resumen_ws.write(row_idx + 1, col_idx, cell_val, fmt)
    
    # Insertar el gráfico basado en resumen1
    chart1 = writer.book.add_chart({'type': 'column'})
    chart1.add_series({
        'name':       'Cantidad de Registros',
        'categories': [resumen_sheet_name, 1, 0, len(resumen1)-1, 0],
        'values':     [resumen_sheet_name, 1, 1, len(resumen1)-1, 1],
        'fill':       {'color': '#5B9BD5'},
    })
    chart1.set_title({'name': 'Cantidad de Registros por Grupo'})
    chart1.set_x_axis({'name': 'Grupo'})
    chart1.set_y_axis({'name': 'Cantidad'})
    chart1.set_style(11)
    
    # Insertar el gráfico en la hoja Resumen en la celda E2 con escalado
    resumen_ws.insert_chart('E2', chart1, {'x_scale': 1.2, 'y_scale': 1.2})
    
    # ========================
    # TABLA 2: Resumen por Municipio
    # ========================
    # Escribir la segunda tabla resumen (resumen2) a partir de la fila siguiente a resumen1
    start_row = len(resumen1) + 3
    # Ajustar ancho de columnas para resumen2
    resumen_ws.set_column(start_row, start_row+4, 20)
    
    # Escribir encabezado de resumen2
    for col_num, value in enumerate(resumen2.columns.tolist()):
         resumen_ws.write(start_row, col_num, value, header_format)
    # Escribir datos de resumen2 (resaltando la fila de totales)
    for i, row in resumen2.iterrows():
         fmt = total_cell_format if i == len(resumen2)-1 else cell_format
         for col_num, cell_val in enumerate(row.tolist()):
             resumen_ws.write(start_row + i + 1, col_num, cell_val, fmt)
    
    # ========================
    # TABLA 3: Resumen de Correos Rechazados
    # ========================
    start_row_correos = start_row + len(resumen2) + 4
    
    # Escribir título
    resumen_ws.write(start_row_correos - 1, 0, "RESUMEN DE CORREOS RECHAZADOS", header_format)
    resumen_ws.merge_range(start_row_correos - 1, 0, start_row_correos - 1, 2, 
                          "RESUMEN DE CORREOS RECHAZADOS", header_format)
    
    # Escribir encabezados
    for col_num, value in enumerate(resumen_correos_rechazados.columns.tolist()):
         resumen_ws.write(start_row_correos, col_num, value, header_format)
    
    # Ajustar ancho de columnas
    resumen_ws.set_column(start_row_correos, 0, 60)  # Motivo de Rechazo
    resumen_ws.set_column(start_row_correos, 1, 15)  # Cantidad
    resumen_ws.set_column(start_row_correos, 2, 15)  # Porcentaje
    
    # Escribir datos (resaltando la fila de totales)
    for i, row in resumen_correos_rechazados.iterrows():
         fmt = total_cell_format if i == len(resumen_correos_rechazados)-1 else cell_format
         for col_num, cell_val in enumerate(row.tolist()):
             resumen_ws.write(start_row_correos + i + 1, col_num, cell_val, fmt)
    
    # Insertar gráfico de correos rechazados
    chart_correos = writer.book.add_chart({'type': 'bar'})
    chart_correos.add_series({
        'name':       'Cantidad de Rechazos',
        'categories': [resumen_sheet_name, start_row_correos + 1, 0, 
                      start_row_correos + min(10, len(resumen_correos_rechazados)-1), 0],
        'values':     [resumen_sheet_name, start_row_correos + 1, 1, 
                      start_row_correos + min(10, len(resumen_correos_rechazados)-1), 1],
        'data_labels': {'value': True},
        'fill':       {'color': '#ED7D31'},
    })
    chart_correos.set_title({'name': 'Top 10 Motivos de Rechazo de Correos'})
    chart_correos.set_x_axis({'name': 'Cantidad'})
    chart_correos.set_y_axis({'name': 'Motivo'})
    chart_correos.set_size({'width': 720, 'height': 576})
    
    # Insertar gráfico a la derecha de la tabla
    resumen_ws.insert_chart(start_row_correos, 4, chart_correos)
    
    # ========================
    # DESCRIPCIÓN FINAL
    # ========================
    # Agregar una descripción debajo de todas las tablas
    desc_row = start_row_correos + len(resumen_correos_rechazados) + 3
    resumen_ws.set_row(desc_row, 50)
    desc_format = writer.book.add_format({
        'italic': True, 
        'font_color': '#595959',
        'text_wrap': True
    })
    descripcion = (
        "La primera tabla muestra la cantidad de registros por cada grupo de validación de datos de contacto. "
        "El primer gráfico ilustra visualmente estas cantidades.\n\n"
        "La segunda tabla detalla, para cada 'Nombre Municipio', cuántos registros corresponden a cada grupo.\n\n"
        f"La tercera tabla muestra el resumen de {len(df_correos_rechazados):,} correos electrónicos rechazados "
        "durante el proceso de validación, con los motivos específicos de cada rechazo. "
        "Para ver el detalle completo, consulte la hoja 'Auditoria_Correos'."
    )
    resumen_ws.write(desc_row, 0, descripcion, desc_format)
    resumen_ws.merge_range(desc_row, 0, desc_row, 3, descripcion, desc_format)
    
    # ========================
    # 🎨 FORMATEAR HOJA DE AUDITORÍA DE CORREOS
    # ========================
    audit_ws = writer.sheets['Auditoria_Correos']
    
    # Aplicar formato a encabezados
    for col_num, value in enumerate(df_correos_rechazados.columns):
        audit_ws.write(0, col_num, value, header_format)
    
    # Ajustar anchos de columna
    audit_ws.set_column('A:A', 15)  # tipo_documento
    audit_ws.set_column('B:B', 20)  # numero_identificacion
    audit_ws.set_column('C:C', 35)  # correo_original
    audit_ws.set_column('D:D', 60)  # motivo_rechazo
    
    # Agregar autofiltro
    audit_ws.autofilter(0, 0, len(df_correos_rechazados), len(df_correos_rechazados.columns) - 1)
    
    # Congelar primera fila
    audit_ws.freeze_panes(1, 0)

print("\n" + "="*80)
print("✅ Archivo Excel guardado exitosamente")
print("="*80)
print(f"📁 Ruta: {Ruta_Salida_Actualización_Datos}")
print(f"\n📊 Hojas creadas:")
print(f"   1. Sin_Datos ({len(df_sin_datos):,} registros)")
print(f"   2. Telefonos ({len(df_telefonos):,} registros)")
print(f"   3. Correo ({len(df_correo):,} registros)")
print(f"   4. OK ({len(df_ok):,} registros)")
print(f"   5. Auditoria_Correos ({len(df_correos_rechazados):,} correos rechazados)")
print(f"   6. Resumen (con 3 tablas y gráficos)")
print("="*80 + "\n")

# 2. NOTIFICAR ACTUALIZACION DE DOCUMENTO

Se organiza la base de datos de los usuarios para evolución de documento

In [ ]:
# Filtrar el DataFrame según las condiciones
Df_Actualizacion_Documento = df_ms_adres[
    (df_ms_adres[33] == 'AC') &  # Columna 33 igual a 'AC'
    (df_ms_adres[42] == 0) &  # Columna 37 igual a 0
    (df_ms_adres[4].isin(['CN', 'RC', 'TI']))  # Columna 4 debe ser NC, RC o TI
]
#Df_Actualizacion_Documento = Df_Actualizacion_Documento.iloc[:, list(range(4, 12)) + ['Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']]

# Obtener los nombres de las columnas correspondientes a los índices del 4 al 11
cols_por_indice = Df_Actualizacion_Documento.columns[4:12].tolist()
# Combinar con las columnas adicionales
columnas_finales = cols_por_indice + ['Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']
Df_Actualizacion_Documento = Df_Actualizacion_Documento.loc[:, columnas_finales]

#Excluye los Presuntos Repetidos PR
# Convertir las columnas a tipo string para asegurar coincidencias exactas
Df_Actualizacion_Documento[5] = Df_Actualizacion_Documento[5].astype(str)
df_PR["N_DOC"] = df_PR["N_DOC"].astype(str)

# Eliminar registros donde las columnas 'celular', 'telefono_1', 'telefono_2' sean vacías o nulas
Df_Actualizacion_Documento = Df_Actualizacion_Documento[
    ~Df_Actualizacion_Documento[['celular', 'telefono_1', 'telefono_2']].isnull().all(axis=1) &
    ~Df_Actualizacion_Documento[['celular', 'telefono_1', 'telefono_2']].eq("").all(axis=1)
]
# Excluir los registros que están en df_PR
Df_Actualizacion_Documento = Df_Actualizacion_Documento[~Df_Actualizacion_Documento[5].isin(df_PR["N_DOC"])]

Df_Actualizacion_Documento.rename(columns={10: "fecha_nacimiento"}, inplace=True)

In [ ]:
# Fecha actual del sistema
current_date = datetime.today()

# Convierte explícitamente la columna a datetime indicando el formato exacto dd/mm/yyyy
Df_Actualizacion_Documento['fecha_nacimiento'] = pd.to_datetime(
    Df_Actualizacion_Documento['fecha_nacimiento'], format='%d/%m/%Y', errors='coerce'
)

# Realiza el cálculo correctamente usando `.dt`
birth = Df_Actualizacion_Documento['fecha_nacimiento']
has_had_birthday = (
    (current_date.month > birth.dt.month) |
    ((current_date.month == birth.dt.month) & (current_date.day >= birth.dt.day))
)

# Calcula la edad exacta
Df_Actualizacion_Documento['edad'] = (
    current_date.year - birth.dt.year - (~has_had_birthday).astype(int)
)

In [ ]:
# Contar el número de registros antes del proceso
initial_count = Df_Actualizacion_Documento.shape[0]

# Crear la nueva columna "Evolución" con valor por defecto "ok"
Df_Actualizacion_Documento["Evolución"] = "ok"

# Definir las condiciones según lo solicitado:
mask_CN = (Df_Actualizacion_Documento[4] == "CN")
mask_RC = (Df_Actualizacion_Documento[4] == "RC") & (Df_Actualizacion_Documento["edad"] > 7)
mask_TI = (Df_Actualizacion_Documento[4] == "TI") & (Df_Actualizacion_Documento["edad"] > 18)

# Asignar los valores correspondientes a la columna "Evolución"
Df_Actualizacion_Documento.loc[mask_CN, "Evolución"] = "evolucion CN a RC"
Df_Actualizacion_Documento.loc[mask_RC, "Evolución"] = "evolución RC a TI"
Df_Actualizacion_Documento.loc[mask_TI, "Evolución"] = "evolución de TI a CC"

# Eliminar los registros en los que "Evolución" es "ok"
df_evolucion = Df_Actualizacion_Documento[Df_Actualizacion_Documento["Evolución"] != "ok"].copy()

# Contar el número de registros después del proceso
final_count = df_evolucion.shape[0]

print(f"Registros antes del proceso: {initial_count}")
print(f"Registros después del proceso: {final_count}")

In [ ]:
evol_values = Df_Actualizacion_Documento["Evolución"].unique()

def safe_sheet_name(name):
    return name if len(name) <= 31 else name[:31]

with pd.ExcelWriter(Ruta_Salida_Actualización_Documento, engine='xlsxwriter') as writer:
    workbook = writer.book
    # Formatos para cabeceras, celdas y totales
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#DCE6F1',
        'border': 1,
        'align': 'center',
        'valign': 'vcenter'
    })
    cell_format = workbook.add_format({
        'border': 1,
        'valign': 'vcenter'
    })
    total_cell_format = workbook.add_format({
        'border': 1,
        'valign': 'vcenter',
        'bold': True,
        'bg_color': '#FFD700'
    })
    
    # Crear hojas por cada grupo de "Evolución", omitiendo la categoría "ok"
    for ev in evol_values:
        if ev == "ok":
            continue
        sheet_name = safe_sheet_name(ev)
        df_group = Df_Actualizacion_Documento[Df_Actualizacion_Documento["Evolución"] == ev]
        df_group.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Crear hoja "Resumen"
    resumen_sheet = "Resumen"
    resumen_ws = workbook.add_worksheet(resumen_sheet)
    
    start_row = 1
    # ====================
    # Tabla: Resumen Evolución (sin la categoría "ok")
    resumen_doc = Df_Actualizacion_Documento[Df_Actualizacion_Documento["Evolución"] != "ok"]\
                    .groupby("Evolución").size().reset_index(name="Cantidad")
    descripciones = {
        "evolucion CN a RC": "Registros que deben evolucionar de CN a RC",
        "evolución RC a TI": "Registros que deben evolucionar de RC a TI",
        "evolución de TI a CC": "Registros que deben evolucionar de TI a CC"
    }
    resumen_doc["Descripción"] = resumen_doc["Evolución"].map(descripciones)
    total_val = resumen_doc["Cantidad"].sum()
    total_row = pd.DataFrame({"Evolución": ["Total"], "Cantidad": [total_val], "Descripción": [""]})
    resumen_doc = pd.concat([resumen_doc, total_row], ignore_index=True)
    
    resumen_ws.write(0, 0, "Resumen Documentos", header_format)
    # Escribir encabezados
    for col_num, col_name in enumerate(resumen_doc.columns):
         resumen_ws.write(start_row, col_num, col_name, header_format)
    # Escribir datos
    for r_idx, row in resumen_doc.iterrows():
         fmt = total_cell_format if row["Evolución"] == "Total" else cell_format
         for c_idx, value in enumerate(row):
              resumen_ws.write(start_row + 1 + r_idx, c_idx, value, fmt)
    nrows_doc = len(resumen_doc) + 1  # Incluye encabezados

    # Agregar gráfica para visualizar el resumen de evolución
    chart = workbook.add_chart({'type': 'column'})
    chart.add_series({
         'name':       'Cantidad',
         'categories': [resumen_sheet, start_row + 1, 0, start_row + nrows_doc - 1, 0],
         'values':     [resumen_sheet, start_row + 1, 1, start_row + nrows_doc - 1, 1],
         'data_labels': {'value': True},
    })
    chart.set_title({'name': 'Resumen Evolución'})
    chart.set_x_axis({'name': 'Evolución'})
    chart.set_y_axis({'name': 'Cantidad'})
    # Insertar la gráfica debajo de la tabla
    resumen_ws.insert_chart(start_row + nrows_doc + 1, 0, chart)
    
    # ====================
    # Tabla: Resumen por Municipio según categorías de "Evolución" (omitiendo "ok")
    resumen_mun = Df_Actualizacion_Documento[Df_Actualizacion_Documento["Evolución"] != "ok"]\
                    .pivot_table(
                        index="Nombre Municipio", 
                        columns="Evolución", 
                        aggfunc='size', 
                        fill_value=0
                    ).reset_index()
    
    # Reordenar columnas para incluir solo las categorías necesarias
    cols_evol = ["evolucion CN a RC", "evolución RC a TI", "evolución de TI a CC"]
    cols_final = ["Nombre Municipio"] + [col for col in cols_evol if col in resumen_mun.columns]
    resumen_mun = resumen_mun[cols_final]
    
    # Agregar fila de totales
    total_vals = resumen_mun.select_dtypes(include='number').sum()
    total_vals["Nombre Municipio"] = "Total"
    total_vals = pd.DataFrame([total_vals])
    resumen_mun = pd.concat([resumen_mun, total_vals], ignore_index=True)
    
    table3_row = start_row + nrows_doc + 2  # Dejar dos filas en blanco
    resumen_ws.write(table3_row - 1, 0, "Resumen por Municipio", header_format)
    # Encabezados de la Tabla 3
    for col_num, col_name in enumerate(resumen_mun.columns):
         resumen_ws.write(table3_row, col_num, col_name, header_format)
    # Datos
    for r_idx, row in resumen_mun.iterrows():
         fmt = total_cell_format if row["Nombre Municipio"] == "Total" else cell_format
         for c_idx, value in enumerate(row):
              resumen_ws.write(table3_row + 1 + r_idx, c_idx, value, fmt)
              
print("Archivo Excel guardado en:", Ruta_Salida_Actualización_Documento)

# 3. USUARIOS ACTIVOS SIN SISBEN (IV)
Notificar a la población, no listado sensal o sin siben para que actualicen sus datos en las tablas de referencia y saquen sisben o validen su listado censal

In [ ]:
# Filtrar el DataFrame según las condiciones
Df_No_Sisben = df_ms_adres


Df_No_Sisben = Df_No_Sisben[
    ~Df_No_Sisben[41].str.contains(
        r"LC\(9\)|LC\(17\)|LC\(1\)|LC\(2\)|LC\(28\)|LC\(16\)|"
        r"SIV\(B\d+\)|SIV\(A\d+\)|SIV\(C\d+\)|"
        r"LC\((?:[^)]*\b(9|2|17|1|22)\b[^)]*)\)", 
        regex=True, 
        na=False
    )
]



Df_No_Sisben = Df_No_Sisben[
    (Df_No_Sisben[2].isnull() | (Df_No_Sisben[2] == '')) &  # Columna 2 vacía o nula
    (Df_No_Sisben[19] != 34) &  # Columna 14 diferente a a población 34
    (Df_No_Sisben[33] == 'AC') &  # Columna 28 igual a 'AC'
    (Df_No_Sisben[42] == 0)  # Columna 37 igual a 0
]

# Obtener los nombres de las columnas correspondientes a los índices del 4 al 11
cols_por_indice = Df_No_Sisben.columns[4:12].tolist() + [Df_No_Sisben.columns[40]]
# Combinar con las columnas adicionales
columnas_finales = cols_por_indice + ['Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']
Df_No_Sisben = Df_No_Sisben.loc[:, columnas_finales]

# Eliminar registros donde las columnas 'celular', 'telefono_1', 'telefono_2' sean vacías o nulas
Df_No_Sisben = Df_No_Sisben[
    ~Df_No_Sisben[['celular', 'telefono_1', 'telefono_2']].isnull().all(axis=1) &
    ~Df_No_Sisben[['celular', 'telefono_1', 'telefono_2']].eq("").all(axis=1)
]

In [ ]:
import pandas as pd

# Crear la tabla resumen por "Nombre Municipio"
resumen_municipios = Df_No_Sisben.groupby('Nombre Municipio').size().reset_index(name='Cantidad')
# Agregar una fila con el total de registros
total_fila = pd.DataFrame({'Nombre Municipio': ['Total'], 'Cantidad': [resumen_municipios['Cantidad'].sum()]})
resumen_municipios = pd.concat([resumen_municipios, total_fila], ignore_index=True)

# Guardar en un archivo Excel
with pd.ExcelWriter(Ruta_Salida_No_Sisben, engine='xlsxwriter') as writer:
    # Guardar el DataFrame original en una hoja
    Df_No_Sisben.to_excel(writer, sheet_name='Datos', index=False)
    
    # Guardar la tabla resumen en otra hoja
    resumen_municipios.to_excel(writer, sheet_name='Resumen', index=False, startrow=1)
    
    # Obtener el workbook y worksheet para aplicar formatos
    workbook = writer.book
    worksheet = writer.sheets['Resumen']
    
    # Formato para la cabecera
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'middle',
        'align': 'center',
        'fg_color': '#DCE6F1',
        'border': 1
    })
    
    # Formato para las celdas
    cell_format = workbook.add_format({
        'border': 1,
        'valign': 'middle'
    })
    
    # Formato para la fila de totales
    total_format = workbook.add_format({
        'bold': True,
        'border': 1,
        'valign': 'middle',
        'fg_color': '#FFD700'
    })
    
    # Aplicar formato a la cabecera
    for col_num, value in enumerate(resumen_municipios.columns):
        worksheet.write(1, col_num, value, header_format)
    
    # Aplicar formato a las celdas
    for row_num, row_data in resumen_municipios.iterrows():
        fmt = total_format if row_data['Nombre Municipio'] == 'Total' else cell_format
        for col_num, cell_value in enumerate(row_data):
            worksheet.write(row_num + 2, col_num, cell_value, fmt)
    
    # Ajustar el ancho de las columnas
    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:B', 15)

print(f"Archivo guardado en: {Ruta_Salida_No_Sisben}")

# 4 NOTIFICACION SISBEN EN OTRO MUNICIPIO

In [ ]:
# Filtrar el DataFrame según las condiciones
Df_Sisben_Otro_Municipio = df_ms_adres

Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio[
    (Df_Sisben_Otro_Municipio[2].isnull() | (Df_Sisben_Otro_Municipio[2] == '')) &  # Columna 2 vacía o nula
    (Df_Sisben_Otro_Municipio[19] == 34) |  # Columna 14 igual a a población 34
    (Df_Sisben_Otro_Municipio[19] == 5) &  # Columna 14 igual a a población 5
    (Df_Sisben_Otro_Municipio[33] == 'AC') &  # Columna 28 igual a 'AC'
    (Df_Sisben_Otro_Municipio[42] == 0)  # Columna 37 igual a 0
]

# Obtener los nombres de las columnas correspondientes a los índices del 4 al 11
cols_por_indice = Df_Sisben_Otro_Municipio.columns[4:12].tolist() + [Df_Sisben_Otro_Municipio.columns[40]]
# Combinar con las columnas adicionales
columnas_finales = cols_por_indice + ['municipio', 'Nombre Departamento', 'Nombre Municipio', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']
Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio.loc[:, columnas_finales]

# Eliminar registros donde las columnas 'celular', 'telefono_1', 'telefono_2' sean vacías o nulas
Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio[
    ~Df_Sisben_Otro_Municipio[['celular', 'telefono_1', 'telefono_2']].isnull().all(axis=1) &
    ~Df_Sisben_Otro_Municipio[['celular', 'telefono_1', 'telefono_2']].eq("").all(axis=1)
]

# Asegurarse de que las columnas de unión sean del mismo tipo
df_Sisben['numeroDocumento'] = df_Sisben['numeroDocumento'].astype(str)
Df_Sisben_Otro_Municipio[5] = Df_Sisben_Otro_Municipio[5].astype(str)

# Realizar el merge para traer la columna "cod_mpio" de df_Sisben a Df_Sisben_Otro_Municipio
Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio.merge(
    df_Sisben[['numeroDocumento', 'cod_mpio']],
    left_on=5,
    right_on='numeroDocumento',
    how='left'
)
# Eliminar la columna duplicada "numeroDocumento" si no es necesaria
Df_Sisben_Otro_Municipio.drop(columns=['numeroDocumento'], inplace=True)

# Filtrar los registros donde "municipio" y "cod_mpio" son diferentes
# Convertir las columnas 'municipio' y 'cod_mpio' a tipo numérico de 5 dígitos
Df_Sisben_Otro_Municipio['municipio'] = Df_Sisben_Otro_Municipio['municipio'].astype(int).apply(lambda x: f"{x:05d}")
Df_Sisben_Otro_Municipio['cod_mpio'] = Df_Sisben_Otro_Municipio['cod_mpio'].fillna(-1).astype(int).apply(lambda x: f"{x:05d}" if x != -1 else "00000")

# Filtrar los registros donde "municipio" y "cod_mpio" son diferentes
Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio[Df_Sisben_Otro_Municipio['municipio'] != Df_Sisben_Otro_Municipio['cod_mpio']]

Df_Sisben_Otro_Municipio.rename(columns={"cod_mpio": "municipio_Sisben"}, inplace=True)
Df_Sisben_Otro_Municipio = Df_Sisben_Otro_Municipio[Df_Sisben_Otro_Municipio['municipio_Sisben'].notnull() & (Df_Sisben_Otro_Municipio['municipio_Sisben'] != "00000")]

In [ ]:
# Crear la tabla resumen por "Nombre Municipio"
resumen_municipios = Df_Sisben_Otro_Municipio.groupby('Nombre Municipio').size().reset_index(name='Cantidad')

# Agregar una fila con el total de registros
total_fila = pd.DataFrame({'Nombre Municipio': ['Total'], 'Cantidad': [resumen_municipios['Cantidad'].sum()]})
resumen_municipios = pd.concat([resumen_municipios, total_fila], ignore_index=True)

# Guardar en un archivo Excel
with pd.ExcelWriter(R_Sisben_OtroMunicipio, engine='xlsxwriter') as writer:
    # Guardar el DataFrame original en una hoja
    Df_Sisben_Otro_Municipio.to_excel(writer, sheet_name='Datos', index=False)
    
    # Guardar la tabla resumen en otra hoja
    resumen_municipios.to_excel(writer, sheet_name='Resumen', index=False, startrow=1)
    
    # Obtener el workbook y worksheet para aplicar formatos
    workbook = writer.book
    worksheet = writer.sheets['Resumen']
    
    # Formato para la cabecera
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'middle',
        'align': 'center',
        'fg_color': '#DCE6F1',
        'border': 1
    })
    
    # Formato para las celdas
    cell_format = workbook.add_format({
        'border': 1,
        'valign': 'middle'
    })
    
    # Formato para la fila de totales
    total_format = workbook.add_format({
        'bold': True,
        'border': 1,
        'valign': 'middle',
        'fg_color': '#FFD700'
    })
    
    # Aplicar formato a la cabecera
    for col_num, value in enumerate(resumen_municipios.columns):
        worksheet.write(1, col_num, value, header_format)
    
    # Aplicar formato a las celdas
    for row_num, row_data in resumen_municipios.iterrows():
        fmt = total_format if row_data['Nombre Municipio'] == 'Total' else cell_format
        for col_num, cell_value in enumerate(row_data):
            worksheet.write(row_num + 2, col_num, cell_value, fmt)
    
    # Ajustar el ancho de las columnas
    worksheet.set_column('A:A', 30)
    worksheet.set_column('B:B', 15)

print(f"Archivo guardado en: {R_Sisben_OtroMunicipio}")